In [7]:
import org.apache.spark.sql.SparkSession

val sparkSession=SparkSession.builder()
  .master("local")
  .appName("Apache SparK App")
  .config("spark.some.config.option", "some-value") 
  .getOrCreate()

import org.apache.spark.sql.SparkSession
sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4d648bda


In [8]:
val flightData2015 = spark
.read
.option("inferSchema", "true")
.option("header", "true")
.csv("../../data/flight-data/csv/2015-summary.csv")

flightData2015: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [9]:
flightData2015.take(3)

res0: Array[org.apache.spark.sql.Row] = Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344])


In [10]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(1) Sort [count#40 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#40 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#51]
   +- FileScan csv [DEST_COUNTRY_NAME#38,ORIGIN_COUNTRY_NAME#39,count#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/media/jose/Repositorio/Git_Hub/Big-Data-Procesing/Spark-The-Definitive-Gu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [11]:
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

res2: Array[org.apache.spark.sql.Row] = Array([United States,Singapore,1], [Moldova,United States,1])


In [12]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [13]:
val sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

val dataFrameWay = flightData2015
.groupBy('DEST_COUNTRY_NAME)
.count()

sqlWay.explain
dataFrameWay.explain

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#80]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#38] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/media/jose/Repositorio/Git_Hub/Big-Data-Procesing/Spark-The-Definitive-Gu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#99]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#38] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/media/jose/Repositorio/Git_Hub/Big-Data-Procesing/Spark-The-Defini

sqlWay: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count(1): bigint]
dataFrameWay: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count: bigint]


In [14]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

res5: Array[org.apache.spark.sql.Row] = Array([370002])


In [15]:
import org.apache.spark.sql.functions.max
flightData2015.select(max("count")).take(1)

import org.apache.spark.sql.functions.max
res6: Array[org.apache.spark.sql.Row] = Array([370002])


In [17]:
val maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



maxSql: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, destination_total: bigint]


In [18]:
import org.apache.spark.sql.functions.desc
flightData2015
.groupBy("DEST_COUNTRY_NAME")
.sum("count")
.withColumnRenamed("sum(count)", "destination_total")
.sort(desc("destination_total"))
.limit(5)
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



import org.apache.spark.sql.functions.desc


In [19]:
flightData2015
.groupBy("DEST_COUNTRY_NAME")
.sum("count")
.withColumnRenamed("sum(count)", "destination_total")
.sort(desc("destination_total"))
.limit(5)
.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#144L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#38,destination_total#144L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[sum(cast(count#40 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#226]
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_sum(cast(count#40 as bigint))])
         +- FileScan csv [DEST_COUNTRY_NAME#38,count#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/media/jose/Repositorio/Git_Hub/Big-Data-Procesing/Spark-The-Definitive-Gu..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




<console>: 3: error: ';' expected but '.' found.